In [ ]:
!pip install langchain langchain-groq langchain-community wikipedia geocoder langsmith -qU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.7/332.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
import geocoder
import os
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_community.tools import TavilySearchResults
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

In [ ]:
location = geocoder.ip('me')

print("Current Location:", location.city)

Current Location: Council Bluffs


In [ ]:
os.environ['MISTRAL_API_KEY'] = userdata.get("MISTRAL_API_KEY")
os.environ['GROQ_API_KEY'] = userdata.get("GROQ_API_KEY")
os.environ['TAVILY_API_KEY'] = userdata.get("TAVILY_API_KEY")
os.environ['LANGSMITH_API_KEY'] = userdata.get("LANGSMITH")

In [ ]:
os.environ["LANGSMITH_TRACING_V2"]="true"
os.environ["LANGSMITH_API_KEY"]="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"
os.environ["LANGSMITH_PROJECT"]="pr-notable-conspiracy-94"
from langsmith import utils
utils.tracing_is_enabled()

True

In [ ]:
tool = TavilySearchResults(max_results=3)

tools = [tool]

# Flight Agent

In [ ]:
flight_prompt = hub.pull("hwchase17/react")
flight_prompt.template = """You are a flight guide assistant, Your task is to extract the location from the user query where the user want to go.
Then you have to provide the flights to the location user wants to go from the closest airport from the user's location. Here is user's location: {user_location}
If the budget is provided then provide the flights in the budget if budget is not provided then provide the flights in any range.
You also have to show the prices, dates(date should be in future timespan of 1 month at most) and class of flights too.
Output in the following format:
Flight 1 | Price of Flight 1 | Date of Flight 1 | Class of Flight 1
Flight 2 | Price of Flight 2 | Date of Flight 2 | Class of Flight 2
Flight 3 | Price of Flight 3 | Date of Flight 3 | Class of Flight 3
Flight 4 | Price of Flight 3 | Date of Flight 4 | Class of Flight 4
Flight 5 | Price of Flight 3 | Date of Flight 5 | Class of Flight 1
Show only 3-5 flights at most and only the list of flights
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat only 1 time)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
llm_flight = ChatGroq( model="llama3-70b-8192")

agent_flight = create_react_agent(llm_flight, tools, flight_prompt)

agent_flight_ex = AgentExecutor(agent=agent_flight, tools=tools, verbose=True,handle_parsing_errors=True)

In [ ]:
query = "Plan a 3-day trip to paris"
flights = agent_flight_ex.invoke({"input":query,"user_location":location.city})



> Entering new AgentExecutor chain...
I need to extract the location from the user query, which is Paris.

Action: tavily_search_results_json
Action Input: "flights from Omaha to Paris"[{'url': 'https://www.expedia.com/lp/flights/oma/cdg/omaha-to-paris', 'content': 'Cheap Flights from Omaha (OMA) to Paris (CDG) start at $463 for one-way and $610 for round trip. Earn your airline miles on top of our rewards!'}, {'url': 'https://www.kayak.com/flight-routes/Omaha-Eppley-Airfield-OMA/Paris-PAR', 'content': 'Find flights to Paris from $560. Fly from Omaha on American Airlines, Iberia, United Airlines and more. Search for Paris flights on KAYAK now to find the best deal.'}, {'url': 'https://www.skyscanner.com/routes/oma/pari/omaha-eppley-airfield-to-paris.html', 'content': 'Paris. $456 per passenger. Departing Tue, Mar 11. One-way flight with British Airways. Outbound indirect flight with British Airways, departing from Omaha Eppley Airfield on Tue, Mar 11, arriving in Paris Charles de Gau

In [ ]:
print(flights['output'])

Flight 1 | $463 | Mar 11 | Economy
Flight 2 | $560 | - | Economy
Flight 3 | $456 | Mar 11 | Economy

Note: The dates and prices are based on the observation from the tavily_search_results_json action. The class of flight is assumed to be Economy as it is not specified in the observation.


## Hotel Agent

In [ ]:
hotel_prompt = hub.pull("hwchase17/react")
hotel_prompt.template = """You are a hotel guide assistant, Your task is to extract the location from the user query where the user want to go.
If the budget is provided then provide the hotels in the budget if budget is not provided then provide the hotels in any range.
You also have to the prices of hotels too.
Output in the following format:
Hotel 1 | Price of Hotel 1
Hotel 2 | Price of Hotel 2
Hotel 3 | Price of Hotel 3
Show only 7-8 hotels at most and only the list
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat only 1 time)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
llm_hotel = ChatGroq(model="llama3-70b-8192")
agent_hotel = create_react_agent(llm_hotel, tools, hotel_prompt)

agent_hotel_ex = AgentExecutor(agent=agent_hotel, tools=tools, verbose=True,handle_parsing_errors=True)

In [ ]:
query = "Plan a 3-day trip to paris"
hotels = agent_hotel_ex.invoke({"input":query})



> Entering new AgentExecutor chain...
I need to extract the location from the user query, which is Paris. 

Action: tavily_search_results_json
Action Input: "hotels in Paris"[{'url': 'https://www.tripadvisor.co.uk/Hotels-g187147-Paris_Ile_de_France-Hotels.html', 'content': 'Popular hotels close to Charles De Gaulle Airport include citizenM Paris Charles de Gaulle Airport Hotel, INNSiDE by Meliá Paris Charles de Gaulle Airport, and Holiday Inn Express Paris - Cdg Airport, an IHG hotel. See the full list: Hotels near Charles De Gaulle Airport.'}, {'url': 'https://www.tripadvisor.com/Hotels-g187147-Paris_Ile_de_France-Hotels.html', 'content': 'Popular hotels close to Charles De Gaulle Airport include citizenM Paris Charles de Gaulle Airport Hotel, INNSiDE by Meliá Paris Charles de Gaulle Airport, and Holiday Inn Express Paris - Cdg Airport, an IHG hotel. See the full list: Hotels near Charles De Gaulle Airport.'}, {'url': 'https://www.cntraveler.com/gallery/the-best-hotels-in-paris', 'c

In [ ]:
print(hotels['output'])

Hotel Elysa-Luxembourg | $175
Prince Albert Montmartre | $200
Elysées Ceramic | $220
Rochester Champs Elysees | $250
Hotel Relais Bosquet by Malone | $280
Novotel Paris Centre Tour Eiffel | $300
Hôtel Lancaster | $316


# Tourist Agent

In [ ]:
tourist_prompt = hub.pull("hwchase17/react")
tourist_prompt.template = """You are a tour guide assistant, Your task is to extract the location from the user query where the user wants to go and
based on the location provide the most popular tourist attractions in that place/city.
Output should be in a list
Tourist Attraction 1
Tourist Attraction 2
Tourist Attraction 3

...
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat only 1 time)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
llm_tourist = ChatGroq(model="llama3-70b-8192")
agent_tourist = create_react_agent(llm_tourist, tools, tourist_prompt)

agent_tourist_ex = AgentExecutor(agent=agent_tourist, tools=tools, verbose=True,handle_parsing_errors=True)

In [ ]:
query = "Plan a 3-day trip to paris"
tour = agent_tourist_ex.invoke({"input":query})



> Entering new AgentExecutor chain...
I need to extract the location from the user query, which is Paris. Now, I need to find the most popular tourist attractions in Paris.

Action: tavily_search_results_json
Action Input: "top tourist attractions in Paris"[{'url': 'https://www.parisdiscoveryguide.com/paris-attractions-top-10.html', 'content': 'Several of these top Paris attractions, such as the soaring\nEiffel Tower, the gorgeous\nSeine River, the monumental\nArc de Triomphe, and even the gleaming white\nSacre Coeur set high on a hilltop, are part of the Paris skyline and easy to spot from many places in the city, whether or not you actually visit.\n Perhaps the most romantic museum in Paris and a top attraction because of its lush sculpture garden, location in a spectacular 18th century rococo mansion, and, of course, the sensual sculpture of two lovers in "The Kiss, the Rodin Museum gives you the opportunity to view the breadth and depth of French sculptor Auguste Rodin\'s boundar

In [ ]:
print(tour['output'])

Eiffel Tower
Arc de Triomphe
Seine River
Sacre Coeur
Rodin Museum
Notre Dame
Louvre Museum
Palais Royal
Palais Garnier
Pont Alexandre III
Tuileries Garden
Pantheon
Catacombs of Paris
Conciergerie
Cluny Museum
Boulevard Saint Germain
